In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import sys
import os

# modify the project path below accordingly
PATH = "/content/drive/MyDrive/HGR/gesture_face_features"

sys.path.append(os.path.join(PATH, "code", "main"))

data_path = os.path.join(PATH, 'samples', 'sign')
feat_path = os.path.join(PATH, 'data', 'feats')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
import torch
from torch.utils.data import DataLoader

import numpy as np

# loading modules from system path
from loader import SignImageFolder, ModelLoader
from participant import participants, filter_participants, filter_targets
from feature import extract_features, extract_model_features
from feature import filter_by_id, get_f_y_pred
from utils import get_predictions, decode_imagenet, delete_keys

# used often
MODEL_POOL = ModelLoader.available_models

In [97]:
dataset = SignImageFolder(data_path)
dataloader = DataLoader(dataset, batch_size=32, num_workers=2)
# have separate dataloader for each model based on input_size
# and recalculate features with extract_features()

In [98]:
feats = extract_model_features(dataloader, feat_path=feat_path)

Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/vgg19_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/resnet50_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/inception_v3_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b0_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b1_feats.pt.
Features loaded from /content/drive/MyDrive/HGR/gesture_face_features/data/feats/efficientnet_b6_feats.pt.


In [108]:
for model_name in MODEL_POOL:
  _, _, _, idx = get_f_y_pred(model_name, feats, dataset.targets)
  label_counts, _ = decode_imagenet(idx)
  print(f'Predictions of {model_name}: {label_counts}')

Predictions of vgg19: {'jersey': 67, 'sweatshirt': 62, 'suit': 34, 'military_uniform': 33, 'cardigan': 4, 'wig': 4, 'abaya': 2}
Predictions of resnet50: {'suit': 113, 'wool': 37, 'jersey': 28, 'wig': 25, 'sunglasses': 2, 'safety_pin': 1}
Predictions of inception_v3: {'jersey': 50, 'wool': 45, 'sweatshirt': 35, 'suit': 27, 'wig': 23, 'Band_Aid': 10, 'velvet': 5, 'sunglasses': 4, 'tench': 3, 'maillot': 2, 'abaya': 1, 'bow_tie': 1}
Predictions of efficientnet_b0: {'sweatshirt': 62, 'suit': 34, 'military_uniform': 34, 'abaya': 32, 'wig': 30, 'jersey': 11, 'wool': 3}
Predictions of efficientnet_b1: {'jersey': 40, 'sweatshirt': 37, 'wool': 37, 'suit': 34, 'abaya': 29, 'necklace': 23, 'perfume': 3, 'sunglasses': 2, 'wig': 1}
Predictions of efficientnet_b6: {'sweatshirt': 66, 'jersey': 38, 'wool': 37, 'suit': 28, 'cardigan': 18, 'wig': 18, 'abaya': 1}


# Model Transferability Estimation

In [103]:
!git clone https://github.com/zhangyikaii/Model-Spider.git

fatal: destination path 'Model-Spider' already exists and is not an empty directory.


In [118]:
# rename cloned directory to model_spider
from mte import score_MTE, available_methods

In [101]:
# MTE uses fc only, so we free RAM
for model_name in MODEL_POOL:
  keys = list(feats[model_name].keys())[:-1]
  delete_keys(feats[model_name], keys)

In [110]:
for method in available_methods:
  score_MTE(feats, dataset.targets, MODEL_POOL, method)
  print()

LogME score for vgg19: 13.308188385500753
LogME score for resnet50: 7.032196750365981
LogME score for inception_v3: 10.980468061676675
LogME score for efficientnet_b0: 10.556121201640433
LogME score for efficientnet_b1: 10.407252203493226
LogME score for efficientnet_b6: 11.352897218778145

NCE score for vgg19: -1.8830607383501146
NCE score for resnet50: -1.9106876169393727
NCE score for inception_v3: -1.7660181098934178
NCE score for efficientnet_b0: -1.8872237662656954
NCE score for efficientnet_b1: -1.8951795760170693
NCE score for efficientnet_b6: -1.8535408778215752

LEEP score for vgg19: -1.9277083486176811
LEEP score for resnet50: -1.9434077278088806
LEEP score for inception_v3: -1.9360991932078393
LEEP score for efficientnet_b0: -1.928917220509696
LEEP score for efficientnet_b1: -1.9320152963534882
LEEP score for efficientnet_b6: -1.9137113130793806

H_Score score for vgg19: 39.069183349609375
H_Score score for resnet50: -14.596046447753906
H_Score score for inception_v3: -815.